# Dataprocessing the Pecan dataset using pandas

### Importing libraries

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
plt.style.use('ggplot')
import matplotlib.dates as dates

# plots directory
figure_directory= 'Discriminative-Sparse-Coding-for-Energy-Disaggregation/figures/'
# change so that all figures have font size defulat as 22
plt.rcParams.update({'font.size': 18})
# figure size always the same
plt.figure(figsize=(16,12))

<Figure size 1152x864 with 0 Axes>

<Figure size 1152x864 with 0 Axes>

In [4]:
import os
import sys
import pandas as pd
from collections import defaultdict

In [97]:

def read_data():
    houses = [135, 275, 580, 774, 898]
    df = pd.DataFrame()
    for house in houses:
        house_df = pd.read_csv('datasets/{}.csv'.format(str(house)))
        house_df.columns = ['localtime', 'use', 'air1', 'furnace1', 'dishwasher1', 'regrigerator1']                         
        appliances_sum = house_df[['air1', 'furnace1', 'dishwasher1', 'regrigerator1']].sum(axis=1)
        house_df['other'] = house_df['use'].subtract(appliances_sum)
        house_df['house'] = house
        house_df = house_df.set_index('house')
        df = pd.concat([df, house_df])
                               
    return df

    
df = read_data()
df.head()

,localtime,use,air1,furnace1,dishwasher1,regrigerator1,other
house,,,,,,,
135,2014-01-01 01:00:00,0.158,0.000,0.009,0.0,0.0,0.149
135,2014-01-01 01:15:00,1.118,1.011,0.070,0.0,0.0,0.037
135,2014-01-01 01:30:00,0.563,0.383,0.038,0.0,0.0,0.142
135,2014-01-01 01:45:00,0.674,0.544,0.043,0.0,0.0,0.087
135,2014-01-01 02:00:00,1.097,0.759,0.060,0.0,0.0,0.278


In [ ]:
d = {}

In [56]:
df=df.assign(new=1)
df.head()

,datetime,use,air1,furnace1,dishwasher1,regrigerator1,other,new
0,2014-01-01 01:00:00,0.158,0.000,0.009,0.0,0.0,0.149,1
1,2014-01-01 01:15:00,1.118,1.011,0.070,0.0,0.0,0.037,1
2,2014-01-01 01:30:00,0.563,0.383,0.038,0.0,0.0,0.142,1
3,2014-01-01 01:45:00,0.674,0.544,0.043,0.0,0.0,0.087,1
4,2014-01-01 02:00:00,1.097,0.759,0.060,0.0,0.0,0.278,1


In [58]:
df.set_index('new')

,datetime,use,air1,furnace1,dishwasher1,regrigerator1,other
new,,,,,,,
1,2014-01-01 01:00:00,0.158,0.000,0.009,0.0,0.0,0.149
1,2014-01-01 01:15:00,1.118,1.011,0.070,0.0,0.0,0.037
1,2014-01-01 01:30:00,0.563,0.383,0.038,0.0,0.0,0.142
1,2014-01-01 01:45:00,0.674,0.544,0.043,0.0,0.0,0.087
1,2014-01-01 02:00:00,1.097,0.759,0.060,0.0,0.0,0.278
1,2014-01-01 02:15:00,0.229,0.000,0.010,0.0,0.0,0.219
1,2014-01-01 02:30:00,0.161,0.000,0.009,0.0,0.0,0.152
1,2014-01-01 02:45:00,1.264,0.845,0.257,0.0,0.0,0.162
1,2014-01-01 03:00:00,0.715,0.522,0.045,0.0,0.0,0.148


In [57]:
grouper = df.groupby('new').value_counts()
grouper.head()

AttributeError: 'DataFrameGroupBy' object has no attribute 'value_counts'

In [27]:
unique = pd.unique(df.index.values.ravel())
full_year = map(lambda x: df[df.index==x].shape[0],unique.tolist())
indices = [i for i, x in enumerate(full_year) if x == 34940]


In [34]:
df[[str('use')]][df[str('use')].index == 1]

,use
1,1.118


In [ ]:


unique = pd.unique(df.index.values.ravel())
# find the houses with the whole year data
full_year = map(lambda x: df[df.index==x].shape[0],unique.tolist())
indices = [i for i, x in enumerate(full_year) if x == 8760]
best_indices = map(lambda x: unique[x],indices)

In [17]:
non_appliances = []
non_appliances.append(df.columns.tolist().index('DateTime'))
# non_appliances.append(df.columns.tolist().index('use'))


[0]

In [10]:
df.head()

,DateTime,grid (135) (kw),air1 (135) (kw),furnace1 (135) (kw),dishwasher1 (135) (kw),refrigerator1 (135) (kw)
0,2014-01-01 01:00:00,0.158,0.000,0.009,0.0,0.0
1,2014-01-01 01:15:00,1.118,1.011,0.070,0.0,0.0
2,2014-01-01 01:30:00,0.563,0.383,0.038,0.0,0.0
3,2014-01-01 01:45:00,0.674,0.544,0.043,0.0,0.0
4,2014-01-01 02:00:00,1.097,0.759,0.060,0.0,0.0
